In [ ]:
import pickle as pkl
import pandas as pd 
import numpy as np

In [ ]:
with open("/media/Datacenter_storage/ramon_dataset_curations/kidney_segmentation/source/debias_seg/data/kits_stu_combined.pkl",'rb') as f: 
   data = pkl.load(f)

In [ ]:
flat_data = [e for row in data for e in row]

In [ ]:
from hashlib import sha224
def  make_hash(path,tag=None): 
    hash_val = sha224(path.encode('utf-8')).hexdigest()
    out_name = f"{hash_val}_{tag}.nii.gz"
    return out_name 

In [ ]:
upload_root="/rodata/mi2c/banerjeei/m261066/kitsdata/"
with open('/media/Datacenter_storage/ramon_dataset_curations/kidney_segmentation/data/copy_items.txt','w')  as f: 
    for row in flat_data:
        vol_path = row['image'] 
        lbl_path = row['label']
        if 'kits'  in vol_path: 
            continue 
        vol_path = vol_path.replace("/media/Datacenter_storage/ramon_dataset_curations/kidney_segmentation/data/STU_noncon_dataset/","./")
        lbl_path = lbl_path.replace("/media/Datacenter_storage/ramon_dataset_curations/kidney_segmentation/data/STU_noncon_dataset/","./")
        #upload_vol = make_hash(vol_path,'vol') 
        #download_vol = make_hash(lbl_path,'lbl')
        #print(f"scp {vol_path} m261066@rohpc10.mayo.edu:{upload_root}{upload_vol}",file=f)
        #print(f"scp {lbl_path} m261066@rohpc10.mayo.edu:{upload_root}{download_vol}",file=f)
        print(f"{vol_path}",file=f)
        print(f"{lbl_path}",file=f)

In [ ]:
new_dsets=  list() 
for i,dset in enumerate(data): 
    new_dset = list() 
    for tup in dset: 
        new_tup  = tup.copy() 
        if tup['phase'] ==1: 
            new_tup['image'] = new_tup['image'].replace('/media/Datacenter_storage/ramon_dataset_curations/kidney_segmentation/source/kits_data_spec/kits21/data/','/rodata/mi2c/banerjeei/m261066/kitsdata/data')
            new_tup['label'] = new_tup['label'].replace('/media/Datacenter_storage/ramon_dataset_curations/kidney_segmentation/source/kits_data_spec/kits21/data/','/rodata/mi2c/banerjeei/m261066/kitsdata/data')
        else: 
            new_tup['image'] = new_tup['image'].replace('/media/Datacenter_storage/ramon_dataset_curations/kidney_segmentation/data/','/rodata/mi2c/banerjeei/m261066/')
            new_tup['label'] = new_tup['label'].replace('/media/Datacenter_storage/ramon_dataset_curations/kidney_segmentation/data/','/rodata/mi2c/banerjeei/m261066/')
            break 
        new_dset.append(new_tup)
    new_dsets.append(new_dset)

In [ ]:
train_df  = pd.DataFrame(data[0])
val = data[1] 
test = data[2]

In [ ]:
train_con = train_df[train_df['phase']==1].copy()
train_non_con = train_df[train_df['phase']==0].copy()

In [ ]:
train_non_con= train_non_con.sample(frac=1,random_state=1996)

In [ ]:
import json 
with open("/media/Datacenter_storage/ramon_dataset_curations/domainadapt_segmentation/configs/step0_baseline/train_baseline.json",'r') as f: 
    template_json =  json.load(f) 

In [ ]:
train_non_con.shape

In [ ]:
ranges = [int(np.floor(frac*train_non_con.shape[0]))+1  for frac in  np.arange(0.0,1.0,0.1)] 

In [ ]:
train_non_con.shape

In [ ]:
ranges[0] = 0

In [ ]:
import numpy as np 
for i,e in enumerate(np.arange(0.0,1,0.1)):
    row_json = template_json.copy() 
    sub_noncon_df = train_non_con.iloc[0:ranges[i],:]
    print(sub_noncon_df.shape)
    #sub_noncon_df = train_non_con.sample(frac=e,random_state=1996)
    tr_df = pd.concat([train_con,sub_noncon_df])
    len(train_non_con)
    tr_l = [row.to_dict() for n,row in tr_df.iterrows()]
    new_pack = (tr_l,val,test) 
    sampling = int(e*10) 
    new_path = f"/media/Datacenter_storage/ramon_dataset_curations/domainadapt_segmentation/data/data_{sampling}.pkl" 
    new_config_path = f"/media/Datacenter_storage/ramon_dataset_curations/domainadapt_segmentation/configs/step0_baseline/subsampling/data_{sampling}.json"
    row_json['log_dir'] = template_json['log_dir'].replace("baseline",f"sampling_{sampling}_p") 
    row_json['log_dir'] = row_json['log_dir'].replace("step0","step5") 
    row_json['data_path'] =new_path 
    print(len(new_pack[0]))
    with open(new_path,'wb') as f:
        pkl.dump(new_pack,f)
    with open(new_config_path,'w') as f: 
        json.dump(row_json,f)